# Read in data from datagolf

In [69]:
df <- read.csv('Data/Euro_preds2.csv',stringsAsFactors=FALSE)
head(df)

X,Name,Name...ID,Salary,make_cut,sample_size,top_10,top_20,top_5,win
<int>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
0,George Coetzee,George Coetzee (15413332),11500,0.886375,150,0.494600,0.655750,0.350950,0.131400
1,Scott Vincent,Scott Vincent (15413334),10500,0.783300,150,0.258550,0.416100,0.152925,0.038925
2,Garrick Higgo,Garrick Higgo (15413333),10900,0.782775,150,0.252000,0.409475,0.148475,0.038000
3,Wade Ormsby,Wade Ormsby (15413337),9700,0.780950,150,0.242525,0.399775,0.141625,0.033725
4,Jamie Donaldson,Jamie Donaldson (15413335),10300,0.757000,150,0.214625,0.361150,0.122125,0.028225
5,Julian Suri,Julian Suri (15413346),8600,0.760175,150,0.214075,0.363575,0.118450,0.027525


In [70]:
#Add column of ones
df['ones'] <- 1
df$ones <- as.integer(df$ones)
head(df)

X,Name,Name...ID,Salary,make_cut,sample_size,top_10,top_20,top_5,win,ones
<int>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
0,George Coetzee,George Coetzee (15413332),11500,0.886375,150,0.494600,0.655750,0.350950,0.131400,1
1,Scott Vincent,Scott Vincent (15413334),10500,0.783300,150,0.258550,0.416100,0.152925,0.038925,1
2,Garrick Higgo,Garrick Higgo (15413333),10900,0.782775,150,0.252000,0.409475,0.148475,0.038000,1
3,Wade Ormsby,Wade Ormsby (15413337),9700,0.780950,150,0.242525,0.399775,0.141625,0.033725,1
4,Jamie Donaldson,Jamie Donaldson (15413335),10300,0.757000,150,0.214625,0.361150,0.122125,0.028225,1
5,Julian Suri,Julian Suri (15413346),8600,0.760175,150,0.214075,0.363575,0.118450,0.027525,1


# Optimize by points_adj

Ref: https://towardsdatascience.com/integer-programming-in-r-33ee6f48a3c8

In [71]:
library('lpSolve')
#library(dplyr)

In [72]:
# Set coefficients of the objective function
f.obj <- df$make_cut

# Set matrix corresponding to coefficients of constraints by rows
#f*dk_salary <= 50000
#f*ones == 6
f.con <- t(data.matrix(df[c('Salary','ones')]))

# Set unequality/equality signs
f.dir <- c("<=",
           "==")

# Set right hand side coefficients
f.rhs <- c(50000,
           6)

# # Variables final values
opt_team <- lp("max", f.obj, f.con, f.dir, f.rhs, int.vec = 1:4, all.bin = TRUE)$solution
df['opt_team'] <- opt_team
subset(df,opt_team==1)

,X,Name,Name...ID,Salary,make_cut,sample_size,top_10,top_20,top_5,win,ones,opt_team
,<int>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,0,George Coetzee,George Coetzee (15413332),11500,0.886375,150,0.494600,0.655750,0.350950,0.131400,1,1
6,5,Julian Suri,Julian Suri (15413346),8600,0.760175,150,0.214075,0.363575,0.118450,0.027525,1,1
8,8,Jens Fahrbring,Jens Fahrbring (15413349),8300,0.736725,150,0.184900,0.327875,0.099250,0.022150,1,1
12,12,Maverick Antcliff,Maverick Antcliff (15413376),7200,0.702925,150,0.149300,0.277050,0.078700,0.016325,1,1
14,14,Pep Angles,Pep Angles (15413377),7200,0.700075,150,0.144925,0.271800,0.076325,0.014975,1,1
18,18,Damien Perrier,Damien Perrier (15413380),7100,0.688300,150,0.137275,0.255725,0.070350,0.013175,1,1


In [73]:
#Find top N teams
num_teams = 7
df_now <- df
to_remove = c('Wade Ormsby')
#to_remove <- c('Steve Stricker')
for (n in to_remove) {
    df_now <- subset(df_now,Name!=n) 
}
#Max sal
max_sal <- 99900
df_now <- subset(df_now,Salary<=max_sal)
min_sal <- 7400
df_now <- subset(df_now,Salary>=min_sal)
base_exposure <- 50
exposure_gap <- 5

#Identify lock
lock = c('George Coetzee')
#lock = c()
num_left <- 6
sal_left <- 50000
if (length(lock) > 0) {
    df_lock <- subset(df_now,Name %in% lock)
    df_now <- subset(df_now,! Name %in% lock)
    sal_used <- sum(df_lock$Salary) 
    num_lock <- length(lock)
    sal_left <- 50000 - sal_used
    num_left <- 6-num_lock
    lock_full_names <- df_lock$Name...ID
} 

for (i in seq(1:num_teams)) {
    # Set coefficients of the objective function
    f.obj <- df_now$make_cut
    f.obj <- df_now$top_20
    #f.obj <- df_now$total_points

    # Set matrix corresponding to coefficients of constraints by rows
    #f*dk_salary <= 50000
    #f*ones == 6
    #f.con <- t(data.matrix(df_now[c('dk_salary','ones','projected_ownership')]))
    f.con <- t(data.matrix(df_now[c('Salary','ones')]))
    
    # Set unequality/equality signs
    f.dir <- c("<=",
               "==")
    
    # Set right hand side coefficients
    f.rhs <- c(sal_left,
               num_left)

    # # Variables final values
    #opt_val <- lp("max", f.obj, f.con, f.dir, f.rhs, int.vec = 1:4, all.bin = TRUE)$objective
    opt_team <- lp("max", f.obj, f.con, f.dir, f.rhs, int.vec = 1:4, all.bin = TRUE)$solution
    df_now['opt_team'] <- opt_team
    df_opt <- subset(df_now,opt_team==1)
    
    #Print current best team
    print(paste('Team ',i,':',sep=""))
    print(paste(df_opt$Name))
    
    #Add best team to df_teams
    if (i==1) {
        df_teams <- df_opt[c('Name...ID','Name','Salary')]
        df_teams['Exposure'] <- base_exposure
    } else {
        df_teams_new <- df_opt[c('Name...ID','Name','Salary')]
        df_teams_new['Exposure'] <- base_exposure - (i-1)*exposure_gap
        df_teams <- rbind(df_teams,df_teams_new)
    }
    
    #Remove current opt team
    df_now <- subset(df_now,opt_team==0)
}

[1] "Team 1:"
[1] "Jens Fahrbring"       "Oscar Lengden"        "Robin Sciot-Siegrist"
[4] "Pedro Oriol"          "Marcel Schneider"    
[1] "Team 2:"
[1] "Julian Suri"                "Bernd Ritthammer"          
[3] "Daan Huizing"               "Jose-Filipe Lima"          
[5] "Kristian Krogh Johannessen"
[1] "Team 3:"
[1] "Martin Simonsen"   "Christopher Mivis" "Marcel Siem"      
[4] "Ryan Evans"        "Enrico Di Nitto"  
[1] "Team 4:"
character(0)


ERROR: Error in `[<-.data.frame`(`*tmp*`, "Exposure", value = 35): replacement has 1 row, data has 0


In [52]:
print(sal_left)

[1] 38500


In [53]:
print(lock_full_names)

[1] "George Coetzee (15413332)"


# Just keep top n teams and sort by salary


In [54]:
teams_to_keep = 2
low_exposure = base_exposure - (teams_to_keep-1)*exposure_gap
df_teams <- subset(df_teams,Exposure >= low_exposure)
df_teams <- df_teams[order(-df_teams$Salary),]
df_teams

,Name...ID,Name,Salary,Exposure
,<chr>,<chr>,<dbl>,<dbl>
6,Julian Suri (15413346),Julian Suri,8600,50
8,Jens Fahrbring (15413349),Jens Fahrbring,8300,50
11,Romain Wattel (15413350),Romain Wattel,8200,45
9,Oscar Lengden (15413361),Oscar Lengden,7600,45
13,Robin Sciot-Siegrist (15413365),Robin Sciot-Siegrist,7500,45
15,Pedro Oriol (15413362),Pedro Oriol,7500,45
19,Marcel Schneider (15413364),Marcel Schneider,7500,45
12,Maverick Antcliff (15413376),Maverick Antcliff,7200,50
14,Pep Angles (15413377),Pep Angles,7200,50


In [55]:
#Same for make_cut
#Find top N teams
num_teams = 7
df_now <- df
#to_remove = c()
#to_remove <- c('Steve Stricker')
for (n in to_remove) {
    df_now <- subset(df_now,Name!=n) 
}
#Max sal
max_sal <- 99900
df_now <- subset(df_now,Salary<=max_sal)
base_exposure <- 50
exposure_gap <- 5

#Identify lock
#lock = c('George Coetzee')
#lock = c()
num_left <- 6
sal_left <- 50000
if (length(lock) > 0) {
    df_lock <- subset(df_now,Name %in% lock)
    df_now <- subset(df_now,! Name %in% lock)
    sal_used <- sum(df_lock$Salary) 
    num_lock <- length(lock)
    sal_left <- 50000 - sal_used
    num_left <- 6-num_lock
    lock_full_names <- df_lock$Name...ID
} 

for (i in seq(1:num_teams)) {
    # Set coefficients of the objective function
    f.obj <- df_now$make_cut
    #f.obj <- df_now$top_20
    #f.obj <- df_now$total_points

    # Set matrix corresponding to coefficients of constraints by rows
    #f*dk_salary <= 50000
    #f*ones == 6
    #f.con <- t(data.matrix(df_now[c('dk_salary','ones','projected_ownership')]))
    f.con <- t(data.matrix(df_now[c('Salary','ones')]))
    
    # Set unequality/equality signs
    f.dir <- c("<=",
               "==")
    
    # Set right hand side coefficients
    f.rhs <- c(sal_left,
               num_left)

    # # Variables final values
    #opt_val <- lp("max", f.obj, f.con, f.dir, f.rhs, int.vec = 1:4, all.bin = TRUE)$objective
    opt_team <- lp("max", f.obj, f.con, f.dir, f.rhs, int.vec = 1:4, all.bin = TRUE)$solution
    df_now['opt_team'] <- opt_team
    df_opt <- subset(df_now,opt_team==1)
    
    #Print current best team
    print(paste('Team ',i,':',sep=""))
    print(paste(df_opt$Name))
    
    #Add best team to df_teams
    if (i==1) {
        df_teams2 <- df_opt[c('Name...ID','Name','Salary')]
        df_teams2['Exposure'] <- base_exposure
    } else {
        df_teams_new <- df_opt[c('Name...ID','Name','Salary')]
        df_teams_new['Exposure'] <- base_exposure - (i-1)*exposure_gap
        df_teams2 <- rbind(df_teams2,df_teams_new)
    }
    
    #Remove current opt team
    df_now <- subset(df_now,opt_team==0)
}

[1] "Team 1:"
[1] "Julian Suri"       "Jens Fahrbring"    "Maverick Antcliff"
[4] "Pep Angles"        "Damien Perrier"   
[1] "Team 2:"
[1] "Oscar Lengden"        "Robin Sciot-Siegrist" "Pedro Oriol"         
[4] "Martin Simonsen"      "Christopher Mivis"   
[1] "Team 3:"
[1] "Scott Vincent"    "Bernd Ritthammer" "Oliver Lindell"   "Dominic Foos"    
[5] "David Boote"     
[1] "Team 4:"
[1] "Gonzalo Fernandez-Castano" "Marcel Schneider"         
[3] "Daan Huizing"              "Roope Kakko"              
[5] "Espen Kofstad"            
[1] "Team 5:"
[1] "Aaron Cockerill"   "Romain Wattel"     "Jose-Filipe Lima" 
[4] "Elias Bertheussen" "Nathan Kimsey"    
[1] "Team 6:"
[1] "Jamie Donaldson" "Ricardo Santos"  "Paul Dunne"      "Rhys Enoch"     
[5] "Anton Karlsson" 
[1] "Team 7:"
[1] "Garrick Higgo"   "Andres Romero"   "Santiago Tarrio" "Mathieu Fenasse"
[5] "Matthew Baldwin"


In [56]:
teams_to_keep = 2
low_exposure = base_exposure - (teams_to_keep-1)*exposure_gap
df_teams2 <- subset(df_teams2,Exposure >= low_exposure)
df_teams2 <- df_teams2[order(-df_teams2$Salary),]
df_teams2

,Name...ID,Name,Salary,Exposure
,<chr>,<chr>,<dbl>,<dbl>
6,Julian Suri (15413346),Julian Suri,8600,50
8,Jens Fahrbring (15413349),Jens Fahrbring,8300,50
17,Christopher Mivis (15413353),Christopher Mivis,7900,45
16,Martin Simonsen (15413356),Martin Simonsen,7800,45
9,Oscar Lengden (15413361),Oscar Lengden,7600,45
13,Robin Sciot-Siegrist (15413365),Robin Sciot-Siegrist,7500,45
15,Pedro Oriol (15413362),Pedro Oriol,7500,45
12,Maverick Antcliff (15413376),Maverick Antcliff,7200,50
14,Pep Angles (15413377),Pep Angles,7200,50


In [57]:
#Whos in one not the other
print(setdiff(df_teams$Name,df_teams2$Name))
print(setdiff(df_teams2$Name,df_teams$Name))

[1] "Romain Wattel"    "Marcel Schneider"
[1] "Christopher Mivis" "Martin Simonsen"  


In [58]:
#Add some more names
more_names = setdiff(df_teams2$Name,df_teams$Name)
# more_names <- c('Rhys Enoch',"Anton Karlsson","Philip Eriksson",
#                "Robin Sciot-Siegrist")
#names_to_add <- setdiff(more_names,df_teams$dk_name)
df_more <- subset(df,Name %in% more_names)[c('Name...ID','Name','Salary')]
df_more['Exposure'] = low_exposure - 5
df_more

,Name...ID,Name,Salary,Exposure
,<chr>,<chr>,<dbl>,<dbl>
16,Martin Simonsen (15413356),Martin Simonsen,7800,40
17,Christopher Mivis (15413353),Christopher Mivis,7900,40


In [59]:
df_teams <- rbind(df_teams,df_more)
df_teams <- df_teams[order(-df_teams$Salary),]
#replace exposure with probabilities
df_teams$Exposure <- df_teams$Exposure / sum(df_teams$Exposure)
df_teams

,Name...ID,Name,Salary,Exposure
,<chr>,<chr>,<dbl>,<dbl>
6,Julian Suri (15413346),Julian Suri,8600,0.09009009
8,Jens Fahrbring (15413349),Jens Fahrbring,8300,0.09009009
11,Romain Wattel (15413350),Romain Wattel,8200,0.08108108
17,Christopher Mivis (15413353),Christopher Mivis,7900,0.07207207
16,Martin Simonsen (15413356),Martin Simonsen,7800,0.07207207
9,Oscar Lengden (15413361),Oscar Lengden,7600,0.08108108
13,Robin Sciot-Siegrist (15413365),Robin Sciot-Siegrist,7500,0.08108108
15,Pedro Oriol (15413362),Pedro Oriol,7500,0.08108108
19,Marcel Schneider (15413364),Marcel Schneider,7500,0.08108108


# Generate lineups

In [60]:
rep('test',6)

[1] "test" "test" "test" "test" "test" "test"

In [61]:
col_names = c('G1','G2','G3','G4','G5','G6')
df_lineups = data.frame('test','test','test','test','test','test',stringsAsFactors=FALSE)
names(df_lineups) <- col_names
df_lineups

G1,G2,G3,G4,G5,G6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
test,test,test,test,test,test


In [62]:
sample(df_teams$Name...ID,size=6,replace=FALSE,prob = df_teams$Exposure)

[1] "Jens Fahrbring (15413349)"    "Julian Suri (15413346)"      
[3] "Romain Wattel (15413350)"     "Oscar Lengden (15413361)"    
[5] "Christopher Mivis (15413353)" "Maverick Antcliff (15413376)"

In [67]:
num_teams <- 44
n <- 0
min_sal <- 49400 - sal_used
max_sal <- 50000 - sal_used
max_own <- 70

#Reste df_teams
df_teams['Actual'] <- 0

#Reset df_lineups
col_names = c('G1','G2','G3','G4','G5','G6')
df_lineups = data.frame('test','test','test','test','test','test',stringsAsFactors=FALSE)
names(df_lineups) <- col_names

while(n<num_teams) {
    #Pick random team
    curr_team <- sample(df_teams$Name...ID,size=num_left,
                        replace=FALSE,prob = df_teams$Exposure)
    df_now <- subset(df_teams,Name...ID %in% curr_team)
    #Get team in order of descending salary
    df_now <- df_now[order(-df_now$Salary),]
    curr_team <- df_now$Name...ID
    curr_sal <- sum(df_now$Salary)
    #curr_own <- sum(df_now$projected_ownership)
    if (curr_sal>=min_sal & curr_sal<=max_sal) {
        curr_team <- c(lock_full_names,curr_team)
        df_lineups <- rbind(df_lineups,curr_team)
        n <- n+1
        if (n%%10 == 0) {
            print(n)
        }
        #Update df_teams
    }
}
head(df_lineups)

[1] 10
[1] 20
[1] 30
[1] 40


G1,G2,G3,G4,G5,G6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
test,test,test,test,test,test
George Coetzee (15413332),Jens Fahrbring (15413349),Romain Wattel (15413350),Robin Sciot-Siegrist (15413365),Maverick Antcliff (15413376),Damien Perrier (15413380)
George Coetzee (15413332),Julian Suri (15413346),Robin Sciot-Siegrist (15413365),Pedro Oriol (15413362),Maverick Antcliff (15413376),Damien Perrier (15413380)
George Coetzee (15413332),Romain Wattel (15413350),Martin Simonsen (15413356),Oscar Lengden (15413361),Marcel Schneider (15413364),Maverick Antcliff (15413376)
George Coetzee (15413332),Julian Suri (15413346),Christopher Mivis (15413353),Marcel Schneider (15413364),Maverick Antcliff (15413376),Pep Angles (15413377)
George Coetzee (15413332),Jens Fahrbring (15413349),Martin Simonsen (15413356),Robin Sciot-Siegrist (15413365),Marcel Schneider (15413364),Pep Angles (15413377)


In [68]:
write.csv(df_lineups,'Lineups/DKEntries.csv')